<a href="https://colab.research.google.com/github/kcezario/conhecimento_acessivel/blob/main/Projeto_Alura_Gemini_Acessibilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conhecimento Acessível: Desvendando o Código
O projeto "Conhecimento Acessível" utiliza o poder da Gemini API do Google para criar um assistente inclusivo, abrindo portas para a educação de pessoas com deficiência visual e não alfabetizados. O código, escrito em Python e executado no Google Colab, implementa diversas funcionalidades que tornam essa visão uma realidade.

In [ ]:
!pip install -q -U google-generativeai
!pip install -q -U SpeechRecognition
!pip install -q -U faster-whisper
!pip install -q -U wikipedia
!pip install -q -U gTTS
!pip install -q -U gTTS pydub

In [ ]:
import re
import numpy as np
import pandas as pd
import google.generativeai as genai
import wikipedia
import time
import inspect
import os
from pydub import AudioSegment
from gtts import gTTS
from faster_whisper import WhisperModel
from io import BytesIO
from base64 import b64decode
from google.colab import output
from google.colab import userdata
from IPython.display import Javascript
from IPython.display import Audio

In [ ]:
API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=API_KEY)

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

In [ ]:
prompt_gemini = {
  'cumprimento': "Você é um assistente simpático, e o texto que você gerar será lido por uma tecnologia de text-to-speech. Cumprimente o usuário. Mantenha o texto curto e sintético, simpático e pergunte o que ele quer saber hoje. Quero apenas a frase",
  'nao_entendi': "Você é um assistente simpático que utiliza text-to-speech. Diga ao usuário que você não entendeu o comando e peça para ele repetir de forma gentil.",
  'despedida': "Você é um assistente simpático que se comunica por text-to-speech. Despeça-se do usuário com um texto curto e gentil.",
  'resumo': "Você é um assistente simpático que usa text-to-speech. Pergunte ao usuário se ele gostaria de ouvir um resumo do artigo.",
  'leitura_completa': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário se ele gostaria de ouvir o artigo completo.",
  '3_perguntas':"Você é um assistente simpático que utiliza text-to-speech. Ofereça ao usuário três opções: ouvir um resumo do artigo, ouvir o artigo completo ou pesquisar outro assunto.",
  'esclarecimento': "Você é um assistente simpático que usa text-to-speech. Pergunte ao usuário se ele deseja que algum ponto específico do artigo seja esclarecido.",
  'proximo_passo': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário o que ele gostaria de fazer agora, mas não ofereça nenhuma opção.",
  'novo_tema': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário sobre qual tema ele gostaria de saber.",
  'erro_tema': "Você é um assistente simpático que utiliza text-to-speech. Peça desculpas por não ter encontrado informações sobre esse tema, peça para o usuário dizer sobre qual outro tema ele gostaria de saber?",
  'perguntas': "Você é um assistente simpático que utiliza text-to-speech. Pergunte ao usuário se ele gostaria de responder a algumas perguntas sobre o tema para testar seus conhecimentos.",
}

In [24]:
# debug = True
debug = False

def log_function_call():
    if debug:
        caller_frame = inspect.currentframe().f_back
        frame_info = inspect.getframeinfo(caller_frame)
        file_name = os.path.basename(frame_info.filename)
        cls_name = caller_frame.f_locals.get('self',
                                             None).__class__.__name__ if 'self' in caller_frame.f_locals else None

        if cls_name:
            print(f"Método {cls_name}.{frame_info.function}.")
        else:
            print(f"Função {frame_info.function}.")

In [ ]:
def gerar_embedding(texto, model=model_embed):
  log_function_call()
  """Gera um embedding para o texto usando a API do Gemini."""
  return genai.embed_content(model=model, content=texto, task_type="RETRIEVAL_QUERY")["embedding"]

In [ ]:
model_embed = "models/embedding-001"
gemini_model = genai.GenerativeModel("gemini-1.0-pro-latest")
gemini_conversa = gemini_model.start_chat(history=[])

df_intencoes = pd.DataFrame({
    'titulo': ["não entendi", "pesquisar", "sair"],
    'conteudo': ["coisas sem sentido", "desejo que você pesquise, me fale sobre, quero saber, quero estudar", "quero sair, adeus, nos vemos, até logo"]
})

df_3_perguntas = pd.DataFrame({
    'titulo': ["resumo", "artigo completo", "outro assunto", "sair"],
    'conteudo': ["quero um resumo, resuma para mim, explique rapidamente", "leia tudo, quero saber mais, comece a leitura", "quero falar de outra coisa, quero entnder sobre outro assunto", "quero sair, adeus, nos vemos, até logo"]
})

df_intencoes['embedding'] = df_intencoes['conteudo'].apply(gerar_embedding)
df_3_perguntas['embedding'] = df_3_perguntas['conteudo'].apply(gerar_embedding)

A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.
A função gerar_embedding - <ipython-input-10-007549d9f9be>.


In [ ]:
whisper_size = 'base'
whisper_model = WhisperModel(whisper_size, device="cpu", compute_type="int8")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
class Chat:

    def __init__(self):
        log_function_call()
        self.input = None
        self.output = None
        self.comando = 'nao_entendi'
        self.prompt = None
        self.intencao = None
        self.tema = None
        self.conversa_iniciada = False
        self.tema_novo = True
        self.audio = None
        self.artigo = None
        self.resumo = None

    def incio(self):
        log_function_call()
        """Inicia o assistente de voz."""
        out = "Iniciando o assistente..."
        print(out)
        self.menu_principal()
        out = "Terminando o programa..."
        print(out)

    def menu_principal(self):
        log_function_call()
        if self.comando != 'pesquisar':
            if self.conversa_iniciada:
                self.comando = 'nao_entendi'
            else:
                self.comando = 'cumprimento'
                self.conversa_iniciada = True
        else:
            self.comando = 'erro_tema'

        voltar = False
        while not voltar:
            self.pegar_resposta()
            self.ouvir_escrever()
            self.intencao = classificar_intencao(self.input, df_intencoes)
            print(f'Intenção: {self.intencao}')
            if self.intencao == 'pesquisar':
                self.tema_novo = True
                self.menu_pesquisa()
            elif self.intencao == 'não entendi':
                pass
            else:
                voltar = True

    def menu_pesquisa(self):
        log_function_call()
        print(f'self.tema_novo - {self.tema_novo}')
        if self.tema_novo:
            self.extrair_tema_wikipedia()
            texto = f"Pesquisando na Wikipédia: {self.tema}"
            self.pesquisar_wikipedia()
            self.comando = '3_perguntas'
            print(texto)
            play(texto)
            self.tema_novo = False
        else:
            self.comando = 'novo_tema'

        voltar = False
        while not voltar:
            self.pegar_resposta()
            self.ouvir_escrever()
            self.intencao = classificar_intencao(self.input, df_3_perguntas)
            print(f'Intenção: {self.intencao}')
            if self.intencao == 'resumo':
                print(self.resumo)
                play(self.resumo)
                self.comando = 'proximo_passo'
            elif self.intencao == 'artigo completo':
                print(self.artigo)
                play(self.artigo)
                self.comando = 'proximo_passo'
            elif self.intencao == 'outro assunto':
                voltar = True
            else:
                break

    def extrair_tema_wikipedia(self):
        log_function_call()
        """Pede ao Gemini para extrair o tema principal da frase para pesquisa na Wikipédia."""
        prompt = f"""Você é um assistente inteligente que ajuda a encontrar informações na Wikipédia.
        Analise a seguinte frase e identifique o assunto principal que o usuário deseja pesquisar: "{self.input}".
        Forneça apenas o assunto principal, formatado como uma única frase, ideal para ser usado como consulta na API da Wikipédia.
        Por exemplo, se a frase for "Gostaria de saber mais sobre a história da pizza", a resposta deve ser "História da pizza".
        Sua resposta:"""

        resposta = gemini_conversa.send_message(prompt)
        tema_wikipedia = resposta.text
        self.tema = tema_wikipedia

    def pesquisar_wikipedia(self):
        log_function_call()
        """Pesquisa um tema na Wikipedia e retorna o título, resumo e conteúdo completo do artigo."""
        try:
            wikipedia.set_lang("pt")
            resultados = wikipedia.search(self.tema)
            if resultados:
                pagina = wikipedia.page(resultados[0])
                self.tema = pagina.title
                self.resumo = pagina.summary
                self.artigo = pagina.content
                self.tema_novo = False
                self.comando = '3_perguntas'
            else:
                out = "Desculpe, não encontrei nenhum artigo na Wikipédia sobre esse tema."
                print(out)
                play(out)
                self.tema = None
                self.tema_novo = True
                self.comando = 'pesquisar'

        except wikipedia.exceptions.PageError:
            out = "Desculpe, não encontrei nenhum artigo na Wikipédia com esse título.", None
            print(out)
            play(out)
            self.tema = None
            self.tema_novo = True
            self.comando = 'pesquisar'

    def falar_gemini(self, texto=""):
        log_function_call()
        expressao = self.prompt + texto
        resposta = gemini_conversa.send_message(expressao)
        self.output = resposta.text
        play(self.output)

    def pegar_resposta(self):
        log_function_call()
        self.prompt = prompt_gemini[self.comando]
        self.falar_gemini()

    def ouvir_escrever(self,sec=3):
        log_function_call()
        audio = self.record()
        self.input = transcrever_audio(audio)
        print(f'self.input - {self.input}')

    def record(self, sec=3):
        log_function_call()
        display(Javascript(RECORD))
        sec += 1
        s = output.eval_js('record(%d)' % (sec*1000))
        play("aguarde")
        print("Pensando...")
        b = b64decode(s.split(',')[1])
        return b

    def pesquisar(self):
        log_function_call()
        self.tema_novo = True
        self.menu_pesquisa()

def transcrever_audio(audio_bytes):
    log_function_call()
    """Transcreve áudio usando faster_whisper, tratando erros de áudio inválido."""
    max_tentativas = 3  # Número máximo de tentativas de transcrição
    tentativa = 1
    while tentativa <= max_tentativas:
        try:
            with BytesIO(audio_bytes) as audio_stream:
                segments, info = whisper_model.transcribe(audio_stream, language="pt")
                texto = ' '.join([segment.text for segment in segments])
                return texto
        except InvalidDataError:
            print(f"Erro de áudio inválido. Tentativa {tentativa} de {max_tentativas}. Aguardando mais tempo...")
            time.sleep(2)  # Aguarda 2 segundos antes de tentar novamente
            tentativa += 1
    return ""  # Retorna vazio se todas as tentativas falharem

def play(text):
    log_function_call()
    text = "   " + text
    tts = gTTS(text, lang='pt')
    tts.save('output.mp3')
    audio_segment = AudioSegment.from_mp3('output.mp3')
    duration_seconds = len(audio_segment) / 1000.0  # duração em segundos
    display(Audio('output.mp3', autoplay=True))
    time.sleep(duration_seconds)

def classificar_intencao(consulta, df, model=model_embed):
    log_function_call()
    """Classifica a intenção do usuário com base na similaridade de embeddings."""
    embedding_da_consulta = gerar_embedding(consulta, model)
    produtos_escalares = np.dot(np.stack(df["embedding"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["titulo"]

In [25]:
chat = Chat()
chat.incio()

Iniciando o assistente...


KeyboardInterrupt: 